# Testing stuff before putting it in the main code

In [7]:
# Testing new way of importing shift data

import pandas as pd
from datetime import datetime
# Read only the relevant columns from the 'Tabellen' sheet
df_raw = pd.read_excel(
    "data/Data tbv rooster automaat v3 new.xlsx",
    sheet_name="Tabellen",
    usecols="E:P",       # Columns E → P
    skiprows=1
)
# Display the first few rows of the dataframe
print(df_raw.head())

  diensten         actie     begin      eind           deskundigheid maandag  \
0       D1       plannen  07:00:00  15:00:00          2. Verzorgende      Ja   
1       D2       plannen  07:30:00  15:00:00          2. Verzorgende      Ja   
2       D3       plannen  07:30:00  14:00:00  3. Medewerker woonzorg      Ja   
3       D4       plannen  08:45:00  12:45:00  3. Medewerker woonzorg      Ja   
4       E1  niet plannen  07:30:00  14:30:00                 99. NVT     Nee   

  dinsdag woensdag donderdag vrijdag zaterdag zondag  
0      Ja       Ja        Ja      Ja       Ja     Ja  
1      Ja       Ja        Ja      Ja       Ja     Ja  
2      Ja       Ja        Ja      Ja       Ja     Ja  
3      Ja       Ja       Nee     Nee      Nee     Ja  
4     Nee      Nee       Nee     Nee      Nee    Nee  


In [11]:
# Standardize column names for convenience
df_raw.columns = [
    "Shifts", "actie", "Begintijd", "Eindtijd", "Deskundigheid",
    "Maandag", "Dinsdag", "Woensdag", "Donderdag", "Vrijdag",
    "Zaterdag", "Zondag"
]

# Keep only rows where actie == 'plannen'
df_shifts = df_raw[df_raw["actie"].str.lower() == "plannen"].copy()
df_shifts = df_shifts.reset_index(drop=True)

# Convert "Ja"/"Nee" → 1/0
day_cols = ["Maandag","Dinsdag","Woensdag","Donderdag","Vrijdag","Zaterdag","Zondag"]
df_shifts[day_cols] = df_shifts[day_cols].replace({"Ja": 1, "Nee": 0})

# Convert time strings into datetime.time
df_shifts["Begintijd"] = pd.to_datetime(df_shifts["Begintijd"], format="%H:%M:%S").dt.time
df_shifts["Eindtijd"] = pd.to_datetime(df_shifts["Eindtijd"], format="%H:%M:%S").dt.time

# Convert deskundigheid field: extract numeric code
# Example: "2. Verzorgende" → 2
df_shifts["Deskundigheid"] = df_shifts["Deskundigheid"].astype(str).str.extract(r"(\d+)").astype(int)

# Convert to list format (your code expects: [2], or sometimes multiple)
df_shifts["Deskundigheid"] = df_shifts["Deskundigheid"].apply(lambda x: [x])

# Compute duration in hours
df_shifts["Duur"] = (
    pd.to_datetime(df_shifts["Eindtijd"].astype(str)) -
    pd.to_datetime(df_shifts["Begintijd"].astype(str))
).dt.total_seconds() / 3600

# Fix negative durations (crosses midnight)
df_shifts.loc[df_shifts["Duur"] < 0, "Duur"] += 24

print("Shift definitions loaded from Excel.")

Shift definitions loaded from Excel.


C:\Users\20192507\AppData\Local\Temp\ipykernel_876\1450219649.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_shifts[day_cols] = df_shifts[day_cols].replace({"Ja": 1, "Nee": 0})
C:\Users\20192507\AppData\Local\Temp\ipykernel_876\1450219649.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df_shifts["Eindtijd"].astype(str)) -
C:\Users\20192507\AppData\Local\Temp\ipykernel_876\1450219649.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df_sh

In [12]:
df_shifts

,Shifts,actie,Begintijd,Eindtijd,Deskundigheid,Maandag,Dinsdag,Woensdag,Donderdag,Vrijdag,Zaterdag,Zondag,Duur
0,D1,plannen,07:00:00,15:00:00,[2],1,1,1,1,1,1,1,8.00
1,D2,plannen,07:30:00,15:00:00,[2],1,1,1,1,1,1,1,7.50
2,D3,plannen,07:30:00,14:00:00,[3],1,1,1,1,1,1,1,6.50
3,D4,plannen,08:45:00,12:45:00,[3],1,1,1,0,0,0,1,4.00
4,FM,plannen,08:00:00,12:30:00,[6],1,1,1,1,1,0,0,4.50
5,GD,plannen,08:00:00,14:00:00,[4],1,1,1,1,1,1,1,6.00
6,KOK,plannen,15:00:00,19:30:00,[5],1,1,1,1,1,1,1,4.50
7,A1,plannen,15:00:00,23:00:00,[2],1,1,1,1,1,1,1,8.00
8,A2,plannen,15:30:00,22:30:00,[3],1,1,1,1,1,1,1,7.00
9,A3,plannen,18:30:00,02:30:00,[3],1,1,1,1,1,1,0,8.00


In [13]:
test_if_in = 'xx'

if 'x' in test_if_in:
    print("Found x")

Found x


In [1]:
print(str(602859-1))

602858
